<a href="https://colab.research.google.com/github/marekrei/bert_text_classification_example/blob/master/bert_text_classification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Text classification example with BERT
# Created by Marek Rei
# Based on https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/chapter3/section4.ipynb
# Training the model for binary sentiment detection, using the SST2 dataset.

# Some settings
# Which pre-trained model to use.
# See https://huggingface.co/models for options.
checkpoint = "bert-base-uncased"

# How much training data to use.
# 1.0 uses the whole training set but it can take a bit of time to train.
train_data_sample_ratio = 0.1

# Example sentence to use
# We print out predictions for this sentence before and after training
example_sentence = "this was by far the best movie of the year"

In [2]:
# Install the necessary libraries
!pip install datasets==3.6 evaluate transformers[sentencepiece]

In [3]:
# Import the libraries
import torch
import evaluate

from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler
from tqdm.auto import tqdm

In [4]:
# Checking whether you are running on CPU or GPU.
# If the output here says "cuda" then it's running on GPU. Otherwise it's probably CPU.
# In order to run your code in Colab on the GPU, go to Edit -> Notebook settings -> Hardware accelerator and set it to "GPU".
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [5]:
# Loading the pretrained model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Load the data
raw_datasets = load_dataset("glue", "sst2")
raw_datasets.cleanup_cache_files()

# Using only a sample of the training data if needed

if train_data_sample_ratio < 1.0:
    num_training_examples = int(train_data_sample_ratio*len(raw_datasets["train"]))
    raw_datasets["train"] = load_dataset("glue", "sst2", split='train[:'+str(num_training_examples)+']')

# Perform tokenization
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Need to remove columns that the model won't know
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

# DataCollatorWithPadding constructs batches that are padded to the length of the longest sentence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

Map:   0%|          | 0/6734 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [7]:
# Printing out the shapes in one batch
example_batch = None
for batch in train_dataloader:
    example_batch = batch
    break

print({k: v.shape for k, v in example_batch.items()})


# Then printing out the loss, output shape and output values from one batch.
outputs = model(**example_batch.to(device))
print("output.loss: ", outputs.loss)
print("output.logits.shape: ", outputs.logits.shape)
print("output.logits: ", outputs.logits)

# Generating predictions for an example sentence.
# Haven't trained the model yet so these will be random.
def print_example_predictions(example_sentence, example_model):
    _e = tokenize_function({"sentence": example_sentence})
    _k = {k: torch.tensor([_e[k]]).to(device) for k in _e}
    model.eval()
    example_outputs = model(**_k)
    example_logits = example_outputs.logits.cpu().detach().numpy()
    example_probabilities = torch.nn.functional.softmax(example_outputs.logits, dim=1).cpu().detach().numpy()
    print(example_probabilities)
    print("Example sentence: ", example_sentence)
    print("Predicted logits: ", example_logits)
    print("Predicted probabilities: ", example_probabilities)
    print("Prediction: ", "negative" if example_probabilities[0][0] > example_probabilities[0][1] else "positive")

print_example_predictions(example_sentence, model)


{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 35]), 'token_type_ids': torch.Size([8, 35]), 'attention_mask': torch.Size([8, 35])}
output.loss:  tensor(0.7101, device='cuda:0', grad_fn=<NllLossBackward0>)
output.logits.shape:  torch.Size([8, 2])
output.logits:  tensor([[-0.1753,  0.1045],
        [-0.1821,  0.2206],
        [-0.2227,  0.2033],
        [-0.2999,  0.1241],
        [-0.1561,  0.2254],
        [-0.1618,  0.2553],
        [-0.1131,  0.4031],
        [-0.2412,  0.2682]], device='cuda:0', grad_fn=<AddmmBackward0>)
[[0.41273746 0.5872626 ]]
Example sentence:  this was by far the best movie of the year
Predicted logits:  [[-0.18039513  0.17226525]]
Predicted probabilities:  [[0.41273746 0.5872626 ]]
Prediction:  positive


In [8]:
# Setting up model training for fine-tuning
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [9]:
# Setting the model to training mode
model.train()

# Running the training
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/2526 [00:00<?, ?it/s]

In [10]:
# Setting the model to evaluation mode
model.eval()

# Running evaluation
metric = evaluate.load("glue", "sst2")
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())

{'accuracy': 0.8887614678899083}


In [11]:
# Getting predictions for the example sentence again, now that we have trained the model
print_example_predictions(example_sentence, model)

[[2.3682101e-04 9.9976319e-01]]
Example sentence:  this was by far the best movie of the year
Predicted logits:  [[-4.02652   4.321449]]
Predicted probabilities:  [[2.3682101e-04 9.9976319e-01]]
Prediction:  positive
